# Import Necessary Libraries

In [1]:
import os
from dotenv import load_dotenv

from typing import Annotated, Literal

from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, ToolMessage

from pydantic import BaseModel
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [2]:
load_dotenv()

True

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
LLM = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [7]:
LLM.invoke("what are you")

AIMessage(content="I am a large language model, trained by Google.  I am an AI that can communicate and generate human-like text in response to a wide range of prompts and questions.  I don't have personal experiences, feelings, or consciousness.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ffff7f62-1207-43cb-9d2e-8bd1ee17883a-0', usage_metadata={'input_tokens': 4, 'output_tokens': 51, 'total_tokens': 55})

### CATEGORIZE_INITIAL_PROMPT generator

In [8]:
categorize_p_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a Message Categorizing Agent of a Financial Technology Company. You are a master at understanding what a customer wants when they send a message, and you are able to categorize it in a very useful way.
            Conduct a comprehensive analysis of the message provided and categorize it into one of the following categories:
            PURCHASE_REQUEST - used when the customer have stated that they want to make a purchase. For example: "hi, i need a 500 Naira airtime", "hello, i want to buy a 500mb data".
            CHAT_ASSISTANT - used when the customer sends a greeting message or general enquiries about finance, investment and how they can improve their savings, also other conversational questions and  messages. For example: "hello, can you tell me how you can be of help to me?", "what is the color of the sun", "how can i increase my monthly savings up to 50%?", "I need new income flow, do i need to learn new skills to get a better job first?"
            TRANSACTION_HISTORY - used when the customer is asking about something related to their transaction history. It can not be about navigation like where to see or find the history, they have to ask about an INFORMATION concerning a particular history, what transaction took place on a date or the status of a transaction etc. For example: "how much did i spend on october 11?", "how many failed transactions do i have today and what is the cause?". "have i ever bought airtime for 08164758458?"
            CUSTOMER_SUPPORT_REQUIRED - used when the customer have stated that they have an issue with a transaction or purchase, something is not working fine with their account or the customer is complaining about an issue with their purchase, our service etc. For example: "what is wrong?, i bought a 500 Naira airtime earlier today and i am yet to receive it in my account, i was debited immediately after the purchase.", "the receiver is yet the receive the amount i transferred 6 hours ago, please help!!!"
            PLATFORM_NAVIGATION - used when a customer is asking for information about a section of the application, they are having issues finding/accessing/operating a particular section of their account or our application or about a product feature, benefit or service etc. For example: "where do i change my payment information, i can not find any link to such page", "where can i see my recent transactions?", "what can this app help me with?", "how do i know if my purchase was successful?".
            Output a single category only, from the types ('PURCHASE_REQUEST', 'CHAT_ASSISTANT', 'TRANSACTION_HISTORY', 'CUSTOMER_SUPPORT_REQUIRED', 'PLATFORM_NAVIGATION')
            e.g:
            'TRANSACTION_HISTORY'
            """
        ),
        ("human", "{initial_prompt}"),
    ]
)

prompt_category_generator = categorize_p_prompt | LLM | StrOutputParser()

MSG = "hi, why are my transactions failing?"
result = prompt_category_generator.invoke({"initial_prompt": MSG})
print(result)

CUSTOMER_SUPPORT_REQUIRED



## CHAT_ASSISTANT generator

In [9]:
chat_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
        """
            You are a Financial Technology Company's chatbot. You are very kind and helpful when replying customers messages.
            If the customer's message is about Finance, Investments, Economics, Savings and other topics related to the helps a customer can need from a financial company, you should go into details and try as much as you can to ensure that the customer gets relevant and sufficient information for their needs.
            If the customer's messages is about helping them increase their income, savings, investments, passive income strategies, and other topics relating to financial stability and frugal lifestyle techniques. Let the customer know you can help them develop a strategy to reach their goal. First ask the customer complete details about their current situation/spending habits etc and develop an efficient strategy that will help them reach their goal.
            If the customer's message is about general daily conversations like greetings, the weather, football, history (non-financial/economic), their health, the world (not related to economy/finance). If you know the answer, simply give them a simple summary and do not try to go deep into details. If you do not know the answer, simply tell the user that your thinking faculty is occupied with a lot of financial thoughts and that you can not afford to disrupt that because that is where you excel. You can conclude by asking them a question about how many percentage of their income goes into savings or ask if they have reached financial freedom or how far they are from reaching financial freedom and what do they think is making the process slow because you can help them with a plan to overcome whatever is in their way. In a nutshell, just ask a question related to a good financial well being and let them know how you can help them reach the goal in the asked question.
            
        """
        ),
        ("human", "{chat_msg}")
    ]
)

chat_assistant_generator = chat_assistant_prompt | LLM | StrOutputParser()

msg = "i need help optimizing the savings"
result = chat_assistant_generator.invoke({"chat_msg": msg})
print(result)

I'd be happy to help you optimize your savings! To give you the best advice, I need some more information about your current financial situation.  Could you please tell me about:

* **Your current income:**  What is your monthly or annual income after taxes?
* **Your current expenses:**  Do you have a budget? If so, could you share a breakdown of your monthly spending (housing, transportation, food, entertainment, etc.)?  If not, we can work on creating one together.
* **Your current savings:** How much money do you currently have in savings accounts, investment accounts, and other liquid assets?
* **Your savings goals:** What are you saving for? (e.g., down payment on a house, retirement, emergency fund, travel).  What is your target amount and timeframe?
* **Your debt:** Do you have any outstanding debts (credit cards, loans, etc.)? If so, what are the balances and interest rates?
* **Your risk tolerance:** How comfortable are you with the possibility of losing money in investments? 

## PURCHASE_REQUEST generator

In [ ]:
purchase_request_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                
            """
        )
    ]
)